In [2]:
%matplotlib inline

/usr/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2017-11-15 20:14:53.981262. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [1]:
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
import pandas as pd
import seaborn as sns
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import make_scorer
from ipywidgets import interactive
from ipykernel.pylab.backend_inline import flush_figures
from xgboost import XGBClassifier

/usr/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2017-11-15 20:14:50.151420. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [3]:
def gini(actual, pred):
    assert (len(actual) == len(pred))
    all = np.asarray(np.c_[actual, pred, np.arange(len(actual))], dtype=np.float)
    all = all[np.lexsort((all[:, 2], -1 * all[:, 1]))]
    totalLosses = all[:, 0].sum()
    giniSum = all[:, 0].cumsum().sum() / totalLosses

    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)


def gini_normalized(actual, pred):
    return gini(actual, pred) / gini(actual, actual)

/usr/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2017-11-15 20:14:55.658610. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


#### Load data 

In [4]:
data = pd.read_csv("../data/train.csv", na_values=[-1, -1.0], index_col="id")

/usr/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2017-11-15 20:14:58.769310. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [5]:
X, y = data.loc[:, data.columns != "target"], data.target

/usr/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2017-11-15 20:15:02.220448. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


#### Preprocess 

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

/usr/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/usr/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2017-11-15 20:15:02.322067. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


####  Random Forest

In [7]:
%%time
model = RandomForestClassifier()
model.fit(X_train.fillna(X_train.median()), y_train)
y_pred = model.predict_proba(X_test.fillna(X_test.median()))[:, 1]
rf_gini = gini_normalized(y_test, y_pred)
print(rf_gini)

0.0692510368699
CPU times: user 13.8 s, sys: 166 ms, total: 13.9 s
Wall time: 13.9 s


#### K-Nearest-Neighbors 

In [8]:
%%time
model = KNeighborsClassifier()
model.fit(X_train.fillna(X_train.median()), y_train)
y_pred = model.predict_proba(X_test.fillna(X_test.median()))[:, 1]
knn_gini = gini_normalized(y_test, y_pred)
print(knn_gini)

0.0277283196577
CPU times: user 14min 46s, sys: 196 ms, total: 14min 46s
Wall time: 14min 47s


#### Neural Networks (MLP) 

In [9]:
%%time
model = MLPClassifier()
model.fit(X_train.fillna(X_train.median()), y_train)
y_pred = model.predict_proba(X_test.fillna(X_test.median()))[:, 1]
nn_gini = gini_normalized(y_test, y_pred)
print(nn_gini)

0.240792374295
CPU times: user 3min 19s, sys: 557 ms, total: 3min 19s
Wall time: 3min 19s


#### Gradient Boosting 

In [10]:
%%time
model = GradientBoostingClassifier()
model.fit(X_train.fillna(X_train.median()), y_train)
y_pred = model.predict_proba(X_test.fillna(X_test.median()))[:, 1]
gb_gini = gini_normalized(y_test, y_pred)
print(gb_gini)

0.272599114768
CPU times: user 2min 51s, sys: 220 ms, total: 2min 51s
Wall time: 2min 51s


#### AdaBoost 

In [11]:
%%time
model = AdaBoostClassifier()
model.fit(X_train.fillna(X_train.median()), y_train)
y_pred = model.predict_proba(X_test.fillna(X_test.median()))[:, 1]
ada_gini = gini_normalized(y_test, y_pred)
print(ada_gini)

0.256556247356
CPU times: user 37.4 s, sys: 120 ms, total: 37.6 s
Wall time: 37.6 s


#### LightGBM 

In [7]:
%%time
model = LGBMClassifier(n_jobs=3)
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_test)[:, -1]
lgbm_gini = gini_normalized(y_test, y_pred)
print(lgbm_gini)

0.242057888767
CPU times: user 5.19 s, sys: 191 ms, total: 5.38 s
Wall time: 2.33 s


#### XGBoost 

In [ ]:
%%time
model = XGBClassifier(n_jobs=3)
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_test)[:, -1]
xg_gini = gini_normalized(y_test, y_pred)
print(xg_gini)

0.271380686266
CPU times: user 1min 58s, sys: 257 ms, total: 1min 59s
Wall time: 40.7 s


#### Support Vector Machines 

In [ ]:
%%time
model = SVC(probability=True, random_state=42)
model.fit(X_train.fillna(X_train.median()), y_train)
y_pred = model.predict_proba(X_test.fillna(X_test.median()))[:, 1]
svm_gini = gini_normalized(y_test, y_pred)
print(svm_gini)

#### Results 

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.bar(["Random Forest", "GradientBoosting", "AdaBoost", "LightGBM", "XGBoost", "KNN", "NNs", "SVM"], [rf_gini, gb_gini, ada_gini, lgbm_gini, xg_gini, knn_gini, nn_gini, svm_gini])